In [23]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig, pipeline
import torch
import pandas as pd
device = "cuda:0"

In [2]:
load_model_name = "../result/solar_2024-02-28_01-10-25/best" # 학습된 모델 불러오기 가능
load_tokenizer_name = "../result/LDCC-SOLAR-10.7B_2024-02-28_01-10-25"

models = AutoModelForCausalLM.from_pretrained(load_model_name, load_in_8bit=True)
tokenizers = AutoTokenizer.from_pretrained(load_tokenizer_name)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [32]:
print(tokenizers.pad_token)
print(tokenizers.eos_token)

</s>
<|im_end|>


In [21]:
input_text = '석고수정이 발생하는 가장 큰 원인은 무엇인가요? 이를 해결하는 방법에 대해 알려주세요.'

with open("../template/ldcc.txt", "r", encoding="utf-8") as file:
    content = file.read()
answer_start_index = content.find("<answer>")
content = content[:answer_start_index]
content = content.replace("<question>", input_text)


input_ids = tokenizers.encode(
            content, padding=False, max_length=128, return_tensors="pt", add_special_tokens=False
        )
print(content)
print(input_ids)

시트, 마루, 벽면, 도배와 같은 건축의 핵심 자재들과 같은 건축 구조 분야에 대한 질문이 있습니다.
Q: 석고수정이 발생하는 가장 큰 원인은 무엇인가요? 이를 해결하는 방법에 대해 알려주세요.
A:
tensor([[28705, 29236, 29404, 28725, 28705, 30103, 31602, 28725, 28705, 46780,
         29565, 28725, 28705, 29599, 30403, 30275, 28705, 30836, 29538, 28705,
         30771, 46620, 29187, 28705, 46757, 31987, 28705, 29294, 30014, 29939,
         29844, 28705, 30836, 29538, 28705, 30771, 46620, 28705, 29779, 29841,
         28705, 30217, 30263, 29148, 28705, 29634, 29282, 28705, 31959, 29710,
         29015, 28705, 29604, 29570, 29194, 29043, 28723,    13, 28824, 28747,
         28705, 31585, 29511, 29151, 29233, 29015, 28705, 30482, 29693, 29136,
         29175, 28705, 29135, 29747, 28705, 31694, 28705, 29816, 29324, 29538,
         28705, 30449, 46827, 29324, 29135, 29517, 28804, 28705, 29015, 29200,
         28705, 29426, 30075, 29136, 29175, 28705, 30240, 30979, 29148, 28705,
         29634, 29426, 28705, 30976, 30710, 29557, 29721, 29517, 28723,    13,
         28741, 287

In [30]:
output_sequences = models.generate(
        input_ids=input_ids.to("cuda"),
        max_length=512,
        temperature=0.9,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

print(tokenizers.decode(output_sequences[0]))

# text_generator = pipeline("text-generation", model=models, tokenizer=tokenizers) # device=0은 CUDA를 의미
# generated_texts = text_generator(content,
#                                  max_length=512, 
#                                  temperature=0.9, 
#                                  top_k=1, 
#                                  top_p=0.9, 
#                                  repetition_penalty=1.2,
#                                  do_sample=True,
#                                  num_return_sequences=1)

# for text in generated_texts:
#     print(text['generated_text'])

시트, 마루, 벽면, 도배와 같은 건축의 핵심 자재들과 같은 건축 구조 분야에 대한 질문이 있습니다.
Q: 석고수정이 발생하는 가장 큰 원인은 무엇인가요? 이를 해결하는 방법에 대해 알려주세요.
A:석고수정이란 석고제작시 불완전히 합성되었거나 토양으로부터 유출된 쇳물등의 이물질이 석고면에 남아있는 상태를 말합니다. 일반적으로 석이나 철이 혼재된 지역의 토양을 사용할 때 발생하기 쉬우며, 석고판 의심 표면에 검은 점이 보이는것으로 나타납니다. 이러한 경우에는 해당 석고를 품질 저하시김으로 선별해서 사용하지않아야 하며, 입수한 토양을 그래파이트로 첨화하여 잘 섞어준 후에 계속 사용할 수 있습니다.<|im_end|>
